In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install spacy_transformers==1.1.4
!pip install spacy==3.3.1

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [ ]:
spacy.__version__

In [ ]:
!nvidia-smi

In [ ]:
cv_data = json.load(open('/content/drive/MyDrive/Projects/--Res_parser/CV-Parsing-using-Spacy-3-master/data/training/train_data.json' , 'r'))

In [ ]:
len(cv_data)

In [ ]:
!python -m spacy init fill-config /content/drive/MyDrive/Projects/--Res_parser/CV-Parsing-using-Spacy-3-master/data/training/base_config.cfg  /content/drive/MyDrive/Projects/--Res_parser/CV-Parsing-using-Spacy-3-master/data/training/config.cfg

In [ ]:
cv_data[0]

In [ ]:
def get_spacy_docs(file, data):
  nlp = spacy.blank('en')
  db = DocBin()

  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity == True:
        continue

      entity_indices = entity_indices + list(range(start, end))

      try:
        span = doc.char_span(start, end, label=label, alignment_mode='strict')
      except:
          continue

      if span is None:
        err_data = str([start, end]) + "    " + str(text)+ "\n"
        file.write(err_data)

      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass


  return db

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.3)

In [ ]:
len(train), len(test)

In [ ]:
file = open('/content/drive/MyDrive/Projects/--Res_parser/CV-Parsing-using-Spacy-3-master/data/training/error.txt', 'w')

db = get_spacy_docs(file, train)
db.to_disk('/content/drive/MyDrive/Projects/--Res_parser/CV-Parsing-using-Spacy-3-master/data/training/train_data.spacy')

db = get_spacy_docs(file, test)
db.to_disk('/content/drive/MyDrive/Projects/--Res_parser/CV-Parsing-using-Spacy-3-master/data/training/test_data.spacy')

file.close()

In [ ]:
len(db.tokens)

In [ ]:
!python -m spacy train /content/drive/MyDrive/Projects/--Res_parser/CV-Parsing-using-Spacy-3-master/data/training/config.cfg --output.output /content/drive/MyDrive/Projects/--Res_parser/CV-Parsing-using-Spacy-3-master/data/output --paths.train /content/drive/MyDrive/Projects/--Res_parser/CV-Parsing-using-Spacy-3-master/data/training/train_data.spacy --paths.dev /content/drive/MyDrive/Projects/--Res_parser/CV-Parsing-using-Spacy-3-master/data/training/test_data.spacy --gpu-id 0



In [ ]:
#Model Test

In [ ]:
nlp = spacy.load('/content/output/model-best')

In [ ]:
doc = nlp('my name is santhosh. I worked at Microsoft. I have 10 years of experience')
for ent in doc.ents:
  print(ent.text, " ->>>>>>", ent.label_)

In [ ]:
!pip install PyMuPDF

In [ ]:
import sys, fitz

In [ ]:
fname = "/content/drive/MyDrive/Resume  (File responses)/Upload your resume here  (File responses)/Sathishkumar3038.pdf"
doc = fitz.open(fname)

In [ ]:
#doc = [page.getText() for page in doc]

In [ ]:
text = " "
for page in doc:
  text = text + str(page.get_text())

In [ ]:
text = text.strip()

In [ ]:
text = ' '.join(text.split())

In [ ]:
text

In [ ]:
doc = nlp(text)
for ent in doc.ents:
  print(ent.text, " ->>>>>> ", ent.label_)